In [ ]:
!pip3 list

In [ ]:
import json
import time
#from numba import jit
import openpyxl
import os
import torch
print("ok!!")

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = 'cuda' if torch.cuda.is_available() else 'cpu'  #選擇gpu或cpu
print(device)

In [ ]:
def write_excel_xlsx(path, sheet_name, value):
    index = len(value)
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = sheet_name
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.cell(row=i+1, column=j+1, value=str(value[i][j]))
    workbook.save(path)

workbook = None
q_list = []
a_list = []
print("ok!!")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os 
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Bert_project_finetuned_V4(多線程,成功)')
print("ok!!")

In [ ]:
!ls

In [ ]:
from google.colab import files
import json
x = json.dumps ('test')
y = x.encode('utf-8')
def read_corpus():
    """
    读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist 里面。 在此过程中，不用对字符换做任何的处理（这部分需要在 Part 2.3里处理）
    qlist = ["问题1"， “问题2”， “问题3” ....]
    alist = ["答案1", "答案2", "答案3" ....]
    务必要让每一个问题和答案对应起来（下标位置一致）
    """
    # TODO 需要完成的代码部分 ...
    
    # 读取question的json
    with open('q_list.json', 'r') as file:
        question = file.read()
        qlist = json.loads(question)

    # 读取answers的json
    with open('a_list.json', 'r') as file:
        answers = file.read()
        alist = json.loads(answers)
    
    assert len(qlist) == len(alist)  # 确保长度一样
    return qlist, alist

corpus=read_corpus()
qlist=corpus[0]
alist=corpus[1]
print("ok!!")

In [ ]:
import time
def Read_stopwords():
    start=time.time()
    fb=open("stopwords/stopwords.txt","r",encoding='utf-8')
    stopwords_list=[]
    for stopwords in fb.readlines():
        stopwords=stopwords.strip("\n")
        stopwords_list.append(stopwords)
    print("elapsed time of python:",time.time()-start)
    return stopwords_list
stopwords_list=Read_stopwords()
print("ok!!")

In [ ]:
def movestopwords(sentence):
    #from nltk.corpus import stopwords
    #from nltk.tokenize import word_tokenize
    import jieba
    q_filtered_list = []
    # 将每一个问题取出来
    for question in sentence:
        
        # 把问题拆解成一个一个单词
        word_tokens = list(jieba.cut_for_search(question))
        
        
        # 过滤掉停用词
        ## 增加stopwords
        #stop_words = set(stopwords.words('chinese'))
        #stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
        #filtered_words = [word for word in word_tokens if word not in stop_words]
        filtered_words =[word for word in word_tokens if word not in stopwords_list]
        #print(filtered_words)

        # 将单词组合成句子
        filtered_sentens = ' '.join(filtered_words)
       
        q_filtered_list += [filtered_sentens]

    return q_filtered_list
print("ok!!")


In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
# 使用bert-base-uncased
tokenizer = BertTokenizer.from_pretrained('model') #.\model
model = BertModel.from_pretrained('model') #.\model
print("ok!!")

In [ ]:
import json
with open('clean_q_word_list.json','r') as flie:
    clean_q_word_list = flie.read()
    clean_q_list = json.loads(clean_q_word_list)
print("ok!!")

In [ ]:
# 句子的分詞
def get_tokenized_text(sentence):
    
    
    '''time test'''
    #計算程序運行時間
    print("      -----------get_tokenized_text---------------    ")
    starttime = time.time()
    print("      starttime:",starttime)
    

    # 加入句子开头标签【CLS】和结尾标签【SEP】
    #marked_text = "[CLS] " + sentence + " [SEP]"
    marked_text = "[CLS] "+str(sentence)+" [SEP]"
    # 把句子拆成多个分词
    tokenized_text_list = tokenizer.tokenize(marked_text)
    
    #print(tokenized_text_list)
    
    
    '''time test'''
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("      endtime:",endtime)
    print('      time elapsed: ' , timeelapsed , ' seconds\n')
    '''
    if workbook:
        global get_tokenized_text_time
        get_tokenized_text_time+=timeelapsed
    '''
    
    return tokenized_text_list
get_tokenized_text("為甚麼是你?")
print("ok!!")

In [ ]:

def get_Segment_ID(i,tokenized_text_list):
    segments_ids = [i] * len(tokenized_text_list)
    
    return segments_ids
print("ok!!")

In [ ]:
#設定單字的ID。使用詞彙表將標記字符串（或標記序列）轉換為單個整數ID（或ID序列）。
def get_indexed_tokens(tokenized_text_list):
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text_list)
    
    return indexed_tokens
#BERT PyTorch要求张量要转成torch张量 Convert inputs to PyTorch tensors
get_tokens_segments_tensor_time=0
def get_tokens_segments_tensor(i,sentence):
    
    #i=args[0]
    #sentence=args[1]
    # 取得句子分词
    tokenized_text = get_tokenized_text(sentence)
    
    #取得indexed_tokens,segments_ids
    indexed_tokens = get_indexed_tokens(tokenized_text)
    segments_ids = get_Segment_ID(i,tokenized_text)
    
    # 转换成pytorch张量
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    
    
    
    return tokens_tensor, segments_tensors




#為了加快運算速度關閉梯度計算
## Torch.no_Grad关闭梯度计算，节省内存，并加快计算速度
def get_encoded_layers(tokens_tensor,segments_tensors):
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
    return(encoded_layers)
get_sentence_embedding_time=0
def get_sentence_embedding(tokenized_text,encoded_layers):
    
   
    
    
    # Convert the hidden state embeddings into single token vectors
    # Holds the list of 12 layer embeddings for each token
    # Will have the shape: [# tokens, # layers, # features]
    token_embeddings = [] 
    # For each token in the sentence...
    for token_i in range(len(tokenized_text)):
  
      # Holds 12 layers of hidden states for each token 
        hidden_layers = [] 
  
      # For each of the 12 layers...
        for layer_i in range(len(encoded_layers)):
            batch_i = 0
    
        # Lookup the vector for `token_i` in `layer_i`
            vec = encoded_layers[layer_i][batch_i][token_i]
        
            hidden_layers.append(vec)
    
        token_embeddings.append(hidden_layers)
        # Sanity check the dimensions:
        #print ("Number of tokens in sequence:", len(token_embeddings))
        #print ("Number of layers per token:", len(token_embeddings[0]))
        
    ## 特征值[number_of_tokens, 3072]，透过torch.cat把向量拼接
    #concatenated_last_4_layers = [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in token_embeddings]
    ## 特征值相加[number_of_tokens, 768]
    #summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings]
    
    ## 平均每个token的倒数第二层，产生一个768长度句向量
    sentence_embedding = torch.mean(encoded_layers[11], 1)
        
   
    
    return sentence_embedding
print("ok!!")

In [ ]:
#通過average pooling單詞向量來實現句子的向量
import numpy as np

# 将句子拆成每一个词，将词向量平均建构句子的向量
X_bert = []
i = 0
initial_numpy = np.zeros((768,))
def get_tokens_segments_tensor_(i,sentence):
    '''time test
    #計算程序運行時間
    print("      -----------get_tokens_segments_tensor---------------    ")
    starttime = time.time()
    print("      starttime:",starttime)
    '''
    # 取得句子分词
    tokenized_text = get_tokenized_text(sentence)
    
    #取得indexed_tokens,segments_ids
    indexed_tokens = get_indexed_tokens(tokenized_text)
    segments_ids = get_Segment_ID(i,tokenized_text)
    
    # 转换成pytorch张量
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    
    '''time test
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("      endtime:",endtime)
    print('      time elapsed: ' , timeelapsed , ' seconds\n')
    
    if workbook:
        global get_tokens_segments_tensor_time
        get_tokens_segments_tensor_time+=timeelapsed
    '''
    return tokens_tensor, segments_tensors
for sentence in clean_q_list:
    # 句子的分词
    tokenized_text = get_tokenized_text(sentence)
    # BERT PyTorch要求张量要转成torch张量 Convert inputs to PyTorch tensors
    tokens_tensor, segments_tensors = get_tokens_segments_tensor_(i,sentence)
    # 取得隐藏层层数
    encoded_layers = get_encoded_layers(tokens_tensor,segments_tensors)

    # 取得单词向量，取平均成句子的向量
    sentence_embedding = get_sentence_embedding(tokenized_text,encoded_layers)
    
    # 把torch张量转成array
    sentence_embedding_array = np.array(sentence_embedding)

    #array全部组合起来
    initial_numpy = np.vstack((initial_numpy,sentence_embedding_array))
bert_vectorizer = initial_numpy


In [ ]:
# bert的vectorizer存成h5py
import h5py
# Create a new file
f = h5py.File('bert_vectorizer.h5', 'w')
f.create_dataset('bert_vectorizer', data=bert_vectorizer)
f.close()

In [ ]:
#建立倒排表
# 读取question的word_total_list.json
import json
with open('clean_q_word_list.json','r') as flie:
    clean_q_word_list = flie.read()
    clean_q_list = json.loads(clean_q_word_list)

# 读取question的不重复单字word_total_list.json
with open('q_word_list.json','r') as flie:
    q_word_total_list = flie.read()
    word_total_list = json.loads(q_word_total_list)
# 大写变成小写
word_total = [item.lower() for item in word_total_list]



invert_index=dict()
k = 0  # 提醒我目前已经建立到第几个数据
#print(word_total)
for b in word_total:
    temp=[]
    
    j = 0
    while j < len(clean_q_list):
        

        field=clean_q_list[j]
        
        split_field=field.split()
        #print(split_field)
        
        if b in split_field:
            temp.append(j)
        #print(temp)
        j += 1
       
    k += 1
    if k % 10000 == 0:
        print("k:",k)
    
    invert_index[b]=temp
    #print(invert[b])
# 全部单词(53157个单词)对应问题的字典存成json
import json
file_name_6 = 'inverted_idx.json'
with open(file_name_6,'w') as file_object:
    json.dump(invert_index,file_object)

In [ ]:
#(1)去除停用词
def q_movestopwords(sentence):
    #time test
    print("----------------q_movestopwords-----------------")
    starttime = time.time()
    print("starttime:",starttime)
    
    
    #from nltk.corpus import stopwords
    #from nltk.tokenize import word_tokenize
    import jieba
    
    # 英文的大写全部改成小写
    #q_list_lower = sentence.lower()    
    # 把问题拆解成一个一个单词
    word_tokens = jieba.cut_for_search(sentence)
    
    # 过滤掉停用词
    ## 增加stopwords
    #stop_words = set(stopwords.words('chinese'))
    #stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
    #filtered_words = [word for word in word_tokens if word not in stop_words]
   
    filtered_words =[word for word in word_tokens if word not in stopwords_list]
    #print(filtered_words)

    # 将单词组合成句子
    filtered_sentens = ' '.join(filtered_words)
    q_filtered_list = [filtered_sentens]
    
    
    '''
    # 过滤掉停用词
    ## 增加stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
    filtered_words = [word for word in word_tokens if word not in stop_words]

    # 将单词组合成句子
    filtered_sentens = ' '.join(filtered_words)       
    q_filtered_list = [filtered_sentens]
    '''
    
    
    #time test
    endtime = time.time()
    timeelapsed=round(endtime-starttime, 4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    return q_filtered_list , filtered_words

In [ ]:
#(3)透过倒排表搜索
def get_inverted_idx(q_total_word):
    
    #time test
    print("----------------get_inverted_idx-----------------")
    starttime = time.time()
    print("starttime:",starttime)
  
    
    
    
    import json
    # 读取word的倒排表字典.json
    with open('inverted_idx.json', 'r') as fp:
        inverted_idx_dic = json.load(fp)
  
    
    # 查找问题的index列表
    q_total_list = []
    for q_word in q_total_word:
    
        # 只找字典中有的单词进行查找
        if q_word in inverted_idx_dic:
            q_total_list += inverted_idx_dic[q_word]

    # 清除重复的问题的index        
    q_total_clean_list = list(set(q_total_list))####


    #time test
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')


    global col
    col+=1
    if workbook:
        table.cell(row=row_,column=col,value=str(timeelapsed))
 


    return q_total_clean_list


####
#(4) 透过index找出所有相对应的问题
def get_total_q(index_q):
    
    #time test
    print("----------------get_total_q-----------------")
    starttime = time.time()
    print("starttime:",starttime)
    
    
    
    # 读取question的word_total_list.json
    import json
    with open('clean_q_word_list.json','r') as flie:
        clean_q_word_list = flie.read()
        clean_q_list = json.loads(clean_q_word_list)
        
    #找到问题与问题集相似的所有问题
    total_q_list = []
    for q in index_q:
        total_q_list += [clean_q_list[q]]
    
    
    
    #time test
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    
    
    
    return total_q_list

In [ ]:
#(5)执行步骤整理
#get_preprocess:藉由一個使用者問的問題，找到(衍生)58個可能相似的問題 
def get_preprocess(sentence):
    
    #time test
    #計算程序運行時間
    starttime = time.time()
  
    
    
    #1.去除停用詞 2.增加相似詞彙 3.倒排表查詢可能問題
    # 整理完的问题集
    clean_q , word_list = q_movestopwords(sentence)
    #print(clean_q)
    #print(word_list)
    # 相似词列表
    simi_list=word_list
    #simi_list = get_related_words(word_list)######
    # 倒排表找出所有可能的问题index
    q_simi_list = get_inverted_idx(simi_list)
    #print("q_simi_list:",q_simi_list)
    
    # 透过index找出所有相对应的问题
    q_total_list = get_total_q(q_simi_list)
    #print("q_total_list:",q_total_list)
    
    
    
    #time test
    print("----------------get_preprocess-----------------")
    print("starttime:",starttime)
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
   

    return q_total_list , q_simi_list



In [ ]:
## Torch.no_Grad关闭梯度计算，节省内存，并加快计算速度
def get_encoded_layers(tokens_tensor,segments_tensors):
    
    
   
    
    
    import torch
    #from pytorch_pretrained_bert import BertTokenizer
    from pytorch_pretrained_bert import BertModel
    #from pytorch_pretrained_bert import BertForMaskedLM
    # 使用bert-base-chinese
    #tokenizer = BertTokenizer.from_pretrained('.\model')#.\model
    model = BertModel.from_pretrained('./model')
    #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #model = BertModel.from_pretrained('bert-base-uncased')

    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
        
    
    return(encoded_layers)

In [ ]:
initial_numpy = np.zeros((768,)) 

#取得Bert 的句子向量并整理成array(模型之架構: 輸入層->隱藏層1->隱藏層2->.....->輸出層)
import numpy as np
import threading                   ##1
class MyThread(threading.Thread):   ##1
    def __init__(self,i,new_clean_q):
        threading.Thread.__init__(self)
        self.i, self.new_clean_q=i,new_clean_q
        
    def run(self):
        for sentence in self.new_clean_q:
            
            #print("sentence:",sentence)
            # 句子的分词
            tokenized_text = get_tokenized_text(sentence)
            
            
            
            # BERT PyTorch要求张量要转成torch张量 Convert inputs to PyTorch tensors
            tokens_tensor, segments_tensors = get_tokens_segments_tensor(i,sentence)
    
            # 取得隐藏层层数
            encoded_layers = get_encoded_layers(tokens_tensor,segments_tensors)
    
            # 取得单词向量，取平均成句子的向量
            sentence_embedding = get_sentence_embedding(tokenized_text,encoded_layers)
        
            # 把torch张量转成array
            sentence_embedding_array = np.array(sentence_embedding[0])
            
            #array全部组合起来
            #lock.acquire()
            global initial_numpy         
            initial_numpy= np.vstack((initial_numpy,sentence_embedding_array))
            #print("initial_numpy:",initial_numpy)

In [ ]:
def get_new_bert(new_clean_q):
#計算程序運行時間
    print("----------------get_new_bert-----------------")
    starttime = time.time()
    print("starttime:",starttime)
    
    #import numpy as np
    # 将句子拆成每一个词，将词向量平均建构句子的向量
    #X_bert = []
    import numpy as np
    #i = 0
    #initial_numpy = np.zeros((768,))
    #new_clean=new_clean_q
    new_clean_q=np.array(new_clean_q)
    
    

    
    t5=[]
    seperated_array=np.array_split(new_clean_q,3)
    for i in range(3):

        tn=MyThread(i,seperated_array[i])
        #print("seperated_array[i]:",seperated_array[i])
        t5.append(tn)
    #print("t5:",t5)
    
    for j in t5:
        j.start()
        j.join()
    
    #t5[0].start()
    #print("initial_numpy:",initial_numpy)
    bert_vectorizer = initial_numpy
    #print("bert_vectorizer:",bert_vectorizer)
    #print("len(bert_vectorizer):",len(bert_vectorizer))
    
    global col
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("\nendtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    return bert_vectorizer
print("ok!!")

In [ ]:
#(7)整理成餘弦函數輸入的格式 (這裡我一共整理成3種文本表示,前兩篇文章為介紹TF-IDF、Word2Vec有興趣的小伙伴可以找一下)
# 输入tfidf、word2vec、bert资料、1次batch的问题量、是哪一种函数(tfidf、word2vec、bert)
def get_cos_list(new_q_list, q_list_batch, model_type):

    #計算程序運行時間
    print("----------------get_cos_list-----------------")
    starttime = time.time()
    print("starttime:",starttime)

    
    import numpy as np
    
    #设定1000笔作batch
    num = 0
    add_num = q_list_batch
    
    #设定空array
    cos_array = np.empty(shape=[0, 1])
    
    # 输入的资料依照是哪一种函数整理成array(分别为询问的问题、全部的问题、共有多少题)
    new_q_array , ask_q_array , array_length = get_arry(new_q_list, model_type)
    #print("new_q_array:",new_q_array)

    while num< array_length:
        if num + add_num  < array_length:
            cos_distance_array = get_cos_distance(ask_q_array, new_q_array[num : num + add_num])
            cos_array = np.append(cos_array , cos_distance_array)
        else:
            cos_distance_array = get_cos_distance(ask_q_array, new_q_array[num : ])
            cos_array = np.append(cos_array , cos_distance_array)
            
        num += add_num
    #print("cos_distant:",cos_array)
    
    

    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    return cos_array   

In [ ]:
#(8)Bert整理成array
def get_arry(data, model_type):

    if model_type == 'bert':
        import numpy as np
        
        new_array = np.array(data)
        ask_q_array = np.array(data[-1:])
        length = len(data)

    else:
        print('no input model')
    
    return new_array,ask_q_array,length

#(9)余弦函数计算
def get_cos_distance(X1, X2):
    
    import tensorflow as tf
    #import numpy as np    
    
    #sess=tf.Session()
    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        with tf.device('/gpu:0'):
            # calculate cos distance between two sets
            # more similar more big
    
            # 新增询问的问题
            X1_matrix = X1
            
            # 全部的问题
            X2_matrix = X2
          
            
            # 计算矩阵型态
            (k,n) = X1_matrix.shape
            (m,n) = X2_matrix.shape
        with tf.device('/gpu:0'):
            
    
            # 求模
            X1_norm = sess.run(tf.sqrt(tf.reduce_sum(tf.square(X1_matrix), axis=1)))
            X2_norm = sess.run(tf.sqrt(tf.reduce_sum(tf.square(X2_matrix), axis=1)))
            # 内积
            X1_X2_muti = sess.run(tf.matmul(X1_matrix, tf.transpose(X2_matrix)))
            X1_X2_norm = sess.run(tf.matmul(tf.reshape(X1_norm,[k,1]),tf.reshape(X2_norm,[1,m])))
            # 计算余弦距离
            #cos_dis = get_cos_distance(X1_X2_muti, X1_X2_norm )
            
            cos_dis = sess.run(tf.math.divide(X1_X2_muti, X1_X2_norm))
            #cos = cos_dis.eval(session=session)

    sess.close()
    return cos_dis



In [ ]:
#(10)找出前index_num個最相似的問题，返回前58個相似問題的index。index_num個最相似的問题<->前58個相似問題中對應的index
def get_top_index(cos_array,index_num):
 
    #計算程序運行時間
    print("----------------get_top_index-----------------")
    starttime = time.time()
    print("starttime:",starttime)
    
    
    import heapq
    # array转list
    cos_list = cos_array.tolist()
    
    # 最大的索引个数(因为list最后就是新增的问题要去掉,所以要多取1个)
    #因為尾巴新增的問題(使用者問得問題)一定是數值最高的(100%=1符合)，故最大索引個數要加1
    top_num_count = index_num+1
    #map(cos_list.index, heapq.nlargest(top_num_count, cos_list)):
    #利用cos_list.index(heapq.nlargest(top_num_count, cos_list))分別找出
    #前index_num對應數值之index
    max_index =map(cos_list.index, heapq.nlargest(top_num_count, cos_list))#max_index=[相對應index, 相對應index]
    #print(max_index)
    #print("heapq.nlargest(top_num_count, cos_list):",heapq.nlargest(top_num_count, cos_list))
    #heapq.nlargest(n, iterable, key=None):
    #从 iterable 所定义的数据集中返回前 n 个最大元素组成的列表。 如
    #果提供了 key 则其应指定一个单参数的函数，用于从 iterable 的每个
    #元素中提取比较键 (例如 key=str.lower)。 等价于: 
    #sorted(iterable, key=key, reverse=True)[:n]。
    
    #产生成list
    top_index_list = list(max_index)
    top_index = top_index_list
    #print("top_idxs:",top_index)
    
    
    endtime = time.time()
    timeelapsed=round(endtime-starttime,4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    
    return top_index

In [ ]:
#(11)找出相对应最有可能的答案
def get_answer(a_index , q_index):

    
    #計算程序運行時間
    print("----------------get_answer-----------------")
    starttime = time.time()
    print("starttime:",starttime)
    
    
    
    import json
    #print("time now:",time.time())
    with open('a_list.json', 'r') as file:
        answers = file.read()
        alist = json.loads(answers)
    
    q_index_list = []
    #print(q_index_list)
    #从新建立好的倒排表list反推可能问题list
    for u in a_index:   #q_index不包括新增的問題(使用者問得問題)，len=58
        if u < len(q_index):  #如果前index_num個相似問題之index在前58個相似問題之
                              #長度內，當超過範圍時(第59個:使用者問的問題)，即不加入
            q_index_list += [q_index[u]]
    #print("q_index_list:",q_index_list)
    

    #从可能问题的list找到相对应的答案list
    top_a = []
    
    for i in q_index_list:
        top_a += [alist[i]]
    #print("alist:",top_a)
    
    
    endtime = time.time()
    timeelapsed=round(endtime-starttime, 4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    
    
    return top_a

In [ ]:
#(12)执行Bert的步骤整理表
#@jit()
def get_top_results_bert(query):
    #計算程序運行時間
    starttime = time.time()
    
    
    
    # top_idxs存放相似度最高的（存在qlist里的）问题的下表 
    # hint: 利用priority queue来找出top results. 思考为什么可以这么做？ 
    
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words). 
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    # 找出与问题与问题集中相似的问题
    preprocess_q, q_index_list = get_preprocess(query) #preprocess_q=[,,,,可能的問題(文字)]   q_index_list=[,,,可能的問題的index]
    # 建立新的bert的句子词向量(单词向量平均)
    q_new_list = preprocess_q + [query]  #加入使用者問的問題，len長度變59
    new_bert = get_new_bert(q_new_list)
    #取得新问题对问题集的Bert余弦函数
    cos_distant = get_cos_list(new_bert, q_list_batch = 1000, model_type = 'bert')
    # 取得问题在问题集中最相近的5个问题
    top_idxs = get_top_index(cos_distant,5)

    #取得最接近的答案list
    alist = get_answer(top_idxs , q_index_list)

    ##显示答案
    #for top_a in alist:
        #print(top_a) 
        
    global row_
    print("----------------get_top_results_bert-----------------")
    print("starttime:",starttime)
    endtime = time.time()
    timeelapsed=round(endtime-starttime, 4)
    print("endtime:",endtime)
    print('time elapsed: ' , timeelapsed , ' seconds\n')
    
    global col
    col+=1
    table.cell(row=row_,column=col,value=str(timeelapsed))
    
    #row_+=1
    #col=1

    return alist  # 返回相似度最高的问题对应的答案，作为TOP5答案


In [ ]:

#c=0
col =1
row_=2
workbook = None
if(workbook==None):  #建立xlsx檔
    write_excel_xlsx("testGPUV4.xlsx","test_table",
                             [["Question","q_movestopwords","get_inverted_idx","get_total_q",
                              "get_preprocess","get_new_bert","get_cos_list","get_top_index",
                              "get_answer","總共","正確答案"]])
workbook = openpyxl.load_workbook("testGPUV4.xlsx") #載入xlsx檔
sheetnames = workbook.get_sheet_names()
table=workbook.get_sheet_by_name(sheetnames[0])
table=workbook.active
table.cell(row=row_,column=col,value="廣州的快速公交運輸系統每多久就會有一輛巴士？")

answer=get_top_results_bert("廣州的快速公交運輸系統每多久就會有一輛巴士？")
print ('bert:',answer)
print('answer:',answer[0])
#workbook.save('testGPUV4.xlsx')